### 1. Load libraries, set tickers and dates

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
start = dt.datetime(2015,1,1) 
end = dt.datetime.now()

In [ ]:
tickers = ['VB', 'VO', 'VV', 'VNQ', 'VEA', 'VWO'] #list securities/funds in portfolio

### 2. Download price data, convert to returns

In [ ]:
df = yf.download(tickers, start, end) 

In [ ]:
df

In [ ]:
prices = df[['Adj Close']]
prices

In [ ]:
returns=prices.pct_change(1).dropna()
returns

### 3. Prepare data for optimization

In [ ]:
returns.describe() #review data - emsure all tickers have the same number of observations in the data

In [ ]:
num_stocks = len(returns.columns) #Create the initial weights for the optimization function 

init_weights = [1/num_stocks] * num_stocks

In [ ]:
rf_rate = .03 #set the risk-free rate for Sharpe Ratio calculation 

In [ ]:
vcv = returns.cov() #create the variance-co-variance matrix (needed for portfolio variance/std. dev. calcs)

In [ ]:
def get_ret_sd_sr(weights): #returns the portfolios expected return, variance, standard deviation, and sharpe ratio in one array
    ret = np.dot(np.transpose(weights), returns.mean()) * 252
    var = np.dot(np.transpose(weights), np.dot(vcv, weights))
    sd = np.sqrt(var) * np.sqrt(252)
    sr = (ret-rf_rate)/sd
    
    return np.array([ret,sd,sr])

In [ ]:
get_ret_sd_sr(init_weights)

In [ ]:
def neg_sharpe(weights): #function that transforms Sharpe Ratio to a negative value since the optimization we're running is a minimization
    return get_ret_sd_sr(weights)[2] * -1

In [ ]:
neg_sharpe(init_weights)

In [ ]:
bounds = tuple((0.05,.333333) for i in range (num_stocks)) #setting the bounds for the weights of the funds - adjust as needed

In [ ]:
cons = ({'type' : 'eq', 'fun': lambda x : np.sum(x) - 1}) #setting the constraint that the portfolio weights must sum to 100%

### 4. Run optimization 

In [ ]:
results = minimize(neg_sharpe, init_weights, bounds=bounds, constraints=cons) #optimization function

In [ ]:
optimized_weights = pd.DataFrame(results['x'].round(3)*100) #converts results to table in percent form with one decimal
optimized_weights.index = returns.columns
optimized_weights

### 5. Review optimial portfolio results 

In [ ]:
opt_weights = optimized_weights.values.tolist() #create list of optimized weights 
opt_weights

In [ ]:
get_ret_sd_sr(opt_weights) #input optimized weights to view optimal portfolio returns, volatility and share ratio

In [ ]:
exp_returns =returns.mean()*252 #view expected returns, based on historal average, for each security 
exp_returns